![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction:

The AIBox-Dist application works to provide cross camera tracking and identification function. One key step to deploy the distributed application is to calibrate the remotely distributed cameras, i.e. to get the monitoring region and relation to the position/view of the capturing cameras. Please refer to the [Setting up the Board and Application Deployment of Kria™ KV260 Vision AI Starter Kit AIBox-Dist](https://xilinx.github.io/kria-apps-docs/index.html) for more detailed info.

This notebook assists the user to capture the image of remote caputring cameras, and provides an interactive way to get the cooridation of the points you picked up as region of interest, which will be used as part of the calibration json file.

# 2. Imports and Initialization


## Import all python modules required for this notebook.

* Import system, util modules

In [ ]:
%matplotlib widget
import sys
import glob
import subprocess
import re
import os
import cv2

* GStreamer related library import.

In [ ]:
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
gi.require_version('GstVideo', '1.0')
gi.require_version('GIRepository', '2.0')
from gi.repository import GObject, GLib, Gst, GstVideo, GLib, GIRepository
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

# 3. Get the Coordinates to Calibrate the System

## Functions Definition

  3 functions are defined:
  * CaptureCam: Capture the image from the remote camera by providing the IP address
  
  * MarkPoints:   Interactively mark the points
  
  * PrintCoor:    Print out the json section of the coordinates of those points marked in the interactive process by running `MarkPoints`

In [ ]:
def CaptureCam(ip):
    str=("rtspsrc location=\"rtsp://{}:554/test\" ! rtpmp2tdepay ! tsparse ! video/mpegts ! tsdemux ! h264parse ! omxh264dec ! videoconvert ! video/x-raw, format=BGR, width=1920, height=1080 ! appsink".format(ip))
    cap=cv2.VideoCapture(str,cv2.CAP_GSTREAMER)
    _,img=cap.read()
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    imgs[ip]=img
    print("Cam {} captured.".format(ip))
    
def MarkPoints(ip):
    import matplotlib.pyplot as plt
    global ind, coors, imgs
    fig,ax=plt.subplots()
    im=ax.imshow(imgs[ip])

    ind=0
    coors[ip]=[]
    
    global updateimg
    updateimg=imgs[ip].copy()
    def Press(event):
        global updateimg, ind, coors
        updateimg=cv2.circle(updateimg,(int(event.xdata), int(event.ydata)), 10, (0,0,255),10)
        updateimg=cv2.putText(updateimg,"{}".format(ind), (int(event.xdata), int(event.ydata)), cv2.FONT_HERSHEY_SIMPLEX, 5, (0,0,255), 5, cv2.LINE_AA)
        coors[ip].append((int(event.xdata), int(event.ydata)))

        ind+=1
        im.set_data(updateimg)
        fig.canvas.draw()


    fig.canvas.mpl_connect('button_press_event', Press)
    plt.show()

def PrintCoor(ip):
    global coors
    print("IP:",ip)
    print("[\n")
    for ind, xy in enumerate(coors[ip]):
        print(" "*4, "{")
        print(" "*8, '"image":{{ \"x\":{}, "y":{} }},'.format(xy[0], xy[1]))
        print(" "*8, '"plan":{ "x": todo, "y": todo }')
        print(" "*4, "}}{}".format('' if ind==len(coors[ip])-1 else ','))
    print("],")

## Variable Definition

The variable `ip` is the IP address of the remote camera, and will be used as the paramemter to the 3 functions defined above.

In [ ]:
ip="192.168.3.124"

`coors` and `imgs` are the variables to save the coordinates of the marked points and the captured images for all the cameras you referred with the `ip` variable

In [ ]:
coors={}
imgs={}

## Capture and Save the Image of the Remote IP Camera

  Please wait until the message "Cam [IP you provide] captured" shows up.

In [ ]:
CaptureCam(ip)

## Interactively Get the Coordinate of the Point of Interest
Run the following cell to get an interactive image of the remote camera with address `ip`. Please click on the image to mark the points to be used as calibration points.

To clear the marked points and redo, please just rerun the cell.

In [ ]:
MarkPoints(ip)

## Print Out the Coordinates of the Points Marked to the Camera with IP Address `ip`

In [ ]:
PrintCoor(ip)

# 4. Summary

After getting the coordinates, they should be filled to the calibration json file which is needed to run the AIBox-Dist application for the present tracking system.

<center>Copyright© 2021 Xilinx</center>